## Enrich Local Film Data

In [41]:
# !pip install pycountry

In [42]:
import re
import pycountry
import pandas as pd

In [43]:
list(pycountry.countries)[42]

Country(alpha_2='CL', alpha_3='CHL', flag='🇨🇱', name='Chile', numeric='152', official_name='Republic of Chile')

In [44]:
pycountry.historic_countries.get(alpha_3='YUG')

Country(alpha_2='YU', alpha_3='YUG', alpha_4='YUCS', comment="had numeric code 890 until the 'Socialist Federal Republic of Yugoslavia' formerly broke apart on 27 April 1992 and the 'Federal Republic of Yugoslavia' was founded", name='Yugoslavia, (Socialist) Federal Republic of', numeric='891', withdrawal_date='2003-07-23')

In [45]:
# Create Country Dataframe

alpha2 = [c.alpha_2 for c in pycountry.countries]
alpha3 = [c.alpha_3 for c in pycountry.countries]
flags = [c.flag for c in pycountry.countries]
country_names = [c.name for c in pycountry.countries]

countries_df = pd.DataFrame(index = None, data = [alpha2, alpha3, flags, country_names]).T
countries_df.columns = ['Alpha 2', 'Alpha 3', 'Flag', 'Country']

# Create Country Lists

alpha2_to_alpha3 = {str(k).upper(): str(v).upper() for k, v in zip(countries_df['Alpha 2'], countries_df['Alpha 3'])}
alpha3_list = [str(x).upper() for x in countries_df['Alpha 3']]
alpha_3_set = set(alpha3_list)
alpha2_list = [str(x).upper() for x in countries_df['Alpha 2']]
country_names = [str(x) for x in countries_df['Country']]

In [46]:
countries_df.sample(5)

,Alpha 2,Alpha 3,Flag,Country
100,HT,HTI,🇭🇹,Haiti
64,DZ,DZA,🇩🇿,Algeria
92,GT,GTM,🇬🇹,Guatemala
221,TM,TKM,🇹🇲,Turkmenistan
88,GQ,GNQ,🇬🇶,Equatorial Guinea


In [47]:
df_mask = countries_df['Country'].str.contains('India')
filtered_df = countries_df[df_mask]
filtered_df

,Alpha 2,Alpha 3,Flag,Country
104,IN,IND,🇮🇳,India
105,IO,IOT,🇮🇴,British Indian Ocean Territory


In [48]:
df = pd.read_csv('../data/films_folder_report.csv', delimiter = "|", index_col=False)

In [49]:
df.head(2)

,CARPETA,Numero de documentos,Nombres de documentos,Número de documentos de tipo video,Nombre de documentos de tipo video,Tamaño (bytes),Tamaño (legible),Formato de video,Numero de subcarpetas,Nombre de subcarpetas
0,1941 [1979] US Steven Spielberg,2,1941 ES.srt; 1941.mkv,1,1941.mkv,1583667553,1.5 GB,mkv,0,NaN
1,2001 A Space Odyssey (1968),5,2001 A Space Odyssey (1968).mp4; Cover.jpg; in...,1,2001 A Space Odyssey (1968).mp4,1463349866,1.4 GB,mp4,0,NaN


### Extract information from the folder name if available

Types of folder names:
- Type 1: title
- Type 2: t.i.t.l.e.YYYY.other.stuff
- Type 3: title [YYYY] COUNTRIES_CODES direction
- Type 4: title [YYYY] COUNTRY_CODE direction
- Type 5: title (YYYY)
- Type 6: title [YYYY] direction
- Type 7: title (YYYY)
- Type 8: title - direction YYYY
- Type 9:
- Type 10:

Information that can be extracted from the folder name:

- Title
- Year
- Direction
- Country

In [50]:
bad_country_names_dict = {
    'AUSTRIA': 'AUT',
    'CESK': 'CZE',
    'COREA': 'KOR',
    'DEN': 'DNK',
    'GER': 'DEU',
    'IN': 'IND',
    'JAP': 'JPN',
    'RUM': 'ROU',
    'SWI': 'CHE',
    'TAIWAN': 'TWN',
    'TV': '',
    'UK': 'GBR',
    'VI': 'vi',
    'YUG': '',
}


alpha3_pattern = re.compile(r'\b(?:' + '|'.join(sorted(alpha3_list, key=len, reverse=True)) + r')\b')
alpha2_pattern = re.compile(r'\b(?:' + '|'.join(sorted(alpha2_list, key=len, reverse=True)) + r')\b')


def detect_alpha_codes(title):
    """
    """
    if not isinstance(title, str):
        return None, None, title

    text = title 
    raw_token = None
    alpha3_final = None

    m3 = alpha3_pattern.search(text)
    if m3:
        raw_token = m3.group()
        alpha3_final = raw_token
        normalized = re.sub(re.escape(raw_token), alpha3_final, text, count=1)
        return alpha3_final, raw_token, normalized

    m2 = alpha2_pattern.search(text)
    if m2:
        raw_token = m2.group()
        if raw_token.isupper() and raw_token in alpha2_to_alpha3:
            alpha3_final = alpha2_to_alpha3[raw_token]
            normalized = re.sub(re.escape(raw_token), alpha3_final, text, count=1)
            return alpha3_final, raw_token, normalized

    for bad, good in bad_country_names_dict.items():
        if re.search(r'\b' + re.escape(bad) + r'\b', text):
            raw_token = bad          
            alpha3_final = good if good else None  
            normalized = re.sub(re.escape(bad), good if good else '', text, count=1)
            return alpha3_final, raw_token, normalized

    return None, None, text


list_year = []
list_raw_title = []
list_raw_direction = []
list_raw_country = []
list_folder_type = []

allow_lowercase_alpha2 = False

for nombre_original in df['CARPETA']:
    nombre = str(nombre_original) 

    res_1 = re.search(r'(\(\d{4}\))', nombre)
    year_parenthesis = str(res_1.group()).strip('(').strip(')') if res_1 else None

    res_2 = re.search(r'(\[\d{4}\])', nombre)
    year_clasp = str(res_2.group()).strip('[').strip(']') if res_2 else None

    year_general = re.findall(r'(\d{4})', nombre)
    year_general = [res_3 for res_3 in year_general if res_3 != '1080']

    if len(year_general) > 1:
        if  year_parenthesis in year_general:
            year_film = year_parenthesis
        elif year_clasp in year_general:            
            year_film = year_clasp
        else:
            year_film = year_general[1]
    elif len(year_general) == 1:
        year_film = year_general[0]
    else:
        year_film = None


    alpha3_found, matched_token, normalized_nombre = detect_alpha_codes(nombre)
    raw_country = alpha3_found          
    raw_country_token = matched_token 


    if year_film == None:
        raw_title = str.title(nombre)
        raw_direction = None
        folder_name_type = 1
        
    elif ' ' not in nombre:
        raw_title = nombre.split(year_film)
        raw_title = raw_title[0].replace('.', ' ').strip()
        raw_title = str.title(raw_title)
        raw_direction = None
        folder_name_type = 2
    
    elif year_film != None and raw_country != None:                     
        raw_title = nombre.split('[' + year_film + ']')[0].strip()

        if type(raw_country_token) == list:
            raw_direction = nombre.split(str(raw_country_token[-1]))
            raw_direction = str(raw_direction[-1]).strip()
            folder_name_type = 3
        else:
            raw_direction = nombre.split(str(raw_country_token))
            raw_direction = str(raw_direction[-1]).strip()
            folder_name_type = 4
        
    elif '[' + year_film + ']' in nombre:                              
        raw_title = nombre.split('[')[0].strip()
        if ']' in nombre[-1]:
            raw_direction = None
            folder_name_type = 5

        else:                                                           
            raw_direction = nombre.split('[' + year_film + ']')[-1].strip()
            folder_name_type = 6
    
    elif '(' + year_film + ')' in nombre:                               
        raw_title = nombre.split('(')[0].strip()
        raw_direction = None
        folder_name_type = 7

    elif '-' in nombre:                                              
        raw_title = nombre.split('-')[0].strip()
        raw_direction = nombre.split('-')[-1].strip()
        folder_name_type = 8

    elif year_film in nombre:
        raw_title = nombre.split(year_film)[0].strip()
        raw_direction = None
        folder_name_type = 9

    else:
        raw_title = str.title(nombre).strip()
        raw_direction = None
        folder_name_type = 10


    raw_direction = raw_direction.replace(year_film, '') if raw_direction else None


    list_year.append(year_film)
    list_raw_title.append(raw_title)
    list_raw_direction.append(raw_direction)
    list_raw_country.append(raw_country)
    list_folder_type.append(folder_name_type)

    del year_film, raw_title, raw_direction, raw_country, folder_name_type


df['Year'] = list_year
df['RAW_TITLE'] = list_raw_title
df['RAW_DIRECTION'] = list_raw_direction
df['RAW_COUNTRY'] = list_raw_country
df['FOLDER_NAME_TYPE'] = list_folder_type

df['Nombre de subcarpetas'] = df['Nombre de subcarpetas'].fillna('No')

df.to_csv('../data/films_folder_extracted.csv', index=False)